# This is a decode-only Transformer 
## It uses subword tokenization pretrained by bert-base-cased
## Moreover, It relies on the articles of CNN DailyMail Dataset
## It's trained using A P100 GPU for 10000 batches
## All hyperparameters are down 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer

In [7]:
!pip install datasets
!pip install transformers

from datasets import load_dataset

# Load the dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

train = dataset['train']
validation = dataset['validation']

# Print the first article and summary
print('Article:', train[0]['article'])
print('Summary:', train[0]['highlights'])

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Article: It's official: U.S. President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria. Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons. The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction." It's a step that is set to turn an international crisis into a fierce domestic political battle. There are key questions looming over the debate: What did U.N. weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but because h

In [8]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [41]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained(
    "bert-base-cased",
    max_vocab_size=100,
    use_fast=True,
)
tokenizer.vocab_size

28996

In [54]:
# Tokenize the strings
tokenized_data = [torch.tensor(tokenizer.encode(text, truncation="longest_first")).to(device) for text in train[0:100000]['article']]

# Print the first encoded article
print(tokenized_data[0])

tensor([  101,  1135,   112,   188,  2078,   131,   158,   119,   156,   119,
         1697, 14319,  7661,  3349,  1644, 11877,  1106, 18678,  1107,  1113,
         2480,  1106,  1329,  1764,  2049,  1107,  7303,   119,  7661,  1850,
          170,  2998,  1106,  1103,  4075,  1104,  1103,  1585,  1105,  3279,
         1113,  4306,  1480,   117,  2005,  1170, 14563,  1115,  1119,  6616,
         1764,  2168,  1222,  8697,  7539,  1110,  1103,  1268,  2585,  1106,
         1321,  1166,  1103,  6351,  1329,  1104,  5297,  3595,   119,  1109,
         3000,  5626,  1121,  7661,  4390,  2757,  1106, 14942,  1103,  1329,
         1104,  1764,  2049,   107,  1106,  1260,  2083,   117, 26499,   117,
         3843,  1105,  1260, 24633,  1103,  3209,  1111,  2174,  2745,  1104,
         5297,  3595,  1137,  1168,  3595,  1104,  3367,  5915,   119,   107,
         1135,   112,   188,   170,  2585,  1115,  1110,  1383,  1106,  1885,
         1126,  1835,  5532,  1154,   170,  9250,  4500,  1741, 

In [57]:
# Flatten the list of lists into a 1D list
tokenized_data = torch.cat(tokenized_data).view(-1)
print(tokenized_data.shape)

torch.Size([47683367])


In [59]:
print(tokenized_data[0:10])

tensor([ 101, 1135,  112,  188, 2078,  131,  158,  119,  156,  119],
       device='cuda:0')


In [60]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 128 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 96
n_head = 6
n_layer = 3
dropout = 0.2
# ------------

torch.manual_seed(1337)


vocab_size = tokenizer.vocab_size
encode = lambda s: tokenizer.encode(s)
decode = lambda l: tokenizer.decode(l)


# Train and test splits
n = int(0.9*len(tokenized_data)) # first 90% will be train, rest val
train_data = tokenized_data[:n]
val_data = tokenized_data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# @torch.no_grad() this line says to pytorch to prevent backprop since we will be evaluating not real training
@torch.no_grad()
def estimate_loss():
    out = {}
    # let model be in evaluation phase so layers like normalization, .. change their behaviour at inference time
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    # back to training phase
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.head_size = head_size
        # I'm creating this Trill variable Trill is not a parameter of the module so in sort of pytorch
        # conventions this is called a buffer it's not a parameter and you have to call it you have to assign it to the module
        # using a register buffer so that creates the trail, the triangle lower triangular Matrix
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        # we can also drop out here when we calculate the basically affinities and after the softmax we can drop out
        # some of those so we can randomly prevent some of the nodes from communicating
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * self.head_size**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # a linear transformation layer that projects the concatenated output from the self.heads module to the original embedding size n_embd.
        self.proj = nn.Linear(n_embd, n_embd)
        
        # a dropout layer that randomly sets some of the output values to zero during training to prevent overfitting.
        # Dropout is something that you can add right before the residual connection back or right before the connection back into the original pathway
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # we run all of the heads in parallel into a list and simply concatenate all of the outputs and we're concatenating over the channel dimension
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        # The module is defined using the PyTorch nn.Sequential class, which allows us to define a sequence of layers that are applied to the input in order.
        # a feedforward neural network module with two linear layers, a ReLU activation function, and a dropout layer. 
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            # a dropout layer that randomly sets some of the output values to zero during training to prevent overfitting.
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        # n_head heads each of head_size-dimensional self attention running in parallel
        self.sa = MultiHeadAttention(n_head, head_size)

        # feedforward neural network purpose: before when we had the multi-headed self-attention only that did the communication, we went way too fast
        # to calculate the logits so the tokens looked at each other but didn't really have a lot of time to think on what they found from the other tokens
        # notice: that the feed forward here when it's applying linear this is on a per token level all the tokens do this independently so the self-attention is the communication and 
        # then once they've gathered all the data now they need to think on that data individually and so that's what feed forward is doing
        self.ffwd = FeedFoward(n_embd)

        # the size of the layer Norm here is n_embd of 32. so when the layer Norm is normalizing our features it is the normalization here
        # happens the mean and the variance are taking over 32 numbers so the batch and the time act as batch Dimensions both of
        # them so this is kind of like a per token transformation that just normalizes the features and makes them a unit mean unit gaussian at initialization
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # this is actually something that slightly departs from the original paper you see that the [ADD and Norm] is applied after the transformation
        # but um in now it is a bit more basically common to apply the layer Norm before the transformation so there's a reshuffling of the layer Norms 
        # so this is called the [pre-norm formulation] and that's the one that we're going to implement as well
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        # take care now embedding size (= n_embd) != vocab size 
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # positional encoding lookup 
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        
        # final layer norm at the end of the transfomer
        self.ln_f = nn.LayerNorm(n_embd) 
        
        # a fully connected (linear) layer by performing a linear transformation on the input tensor
        # with a weight matrix of size (n_embd, vocab_size) and adding a bias vector of size (vocab_size,)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        
        # now total embedding = token embedding + positional embedding
        x = tok_emb + pos_emb # (B,T,C)
        
        # pass x into
        x = self.blocks(x) # (B,T,C)
        # pass x into
        x = self.ln_f(x) # (B,T,C)

        # logits is the ouput of the fully connected (linear) layer now given input x
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):

            # because now we're using positional embeddings we can never have more than block size coming in because if idx is
            # more than block size then our position embedding table is going to run out of scope because it only has embeddings for up to block size 
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


cuda
5.93722 M parameters
step 0: train loss 10.4485, val loss 10.4551
step 500: train loss 6.1170, val loss 6.6965
step 1000: train loss 5.6854, val loss 6.3357
step 1500: train loss 5.4781, val loss 6.1573
step 2000: train loss 5.3386, val loss 6.0150
step 2500: train loss 5.2390, val loss 5.9136
step 3000: train loss 5.1496, val loss 5.8523
step 3500: train loss 5.0918, val loss 5.7942
step 4000: train loss 5.0422, val loss 5.7260
step 4500: train loss 4.9925, val loss 5.6694
step 5000: train loss 4.9503, val loss 5.6540
step 5500: train loss 4.9195, val loss 5.6196
step 6000: train loss 4.8855, val loss 5.5988
step 6500: train loss 4.8559, val loss 5.5542
step 7000: train loss 4.8326, val loss 5.5401
step 7500: train loss 4.8103, val loss 5.5172
step 8000: train loss 4.7855, val loss 5.4954
step 8500: train loss 4.7679, val loss 5.4752
step 9000: train loss 4.7482, val loss 5.4629
step 9500: train loss 4.7345, val loss 5.4540
step 9999: train loss 4.7196, val loss 5.4360
[PAD] migh